In [1]:
# %%
import datetime
import torchaudio
from data import Category, MyDataset
import analysis as an
from dotenv import load_dotenv
from matplotlib import pyplot as plt
import gc
import tqdm
import warnings
warnings.filterwarnings('ignore')
load_dotenv()

True

In [2]:
base_dir = '\\\\10.166.168.123\\典型城市声纹数据库-标签过'
category_list = ['自然噪声', '社会噪声', '建筑施工噪声', '工业噪声', '交通噪声']

In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: reviy (esil). Use `wandb login --relogin` to force relogin


True

In [4]:
cols = ["file", "audio", "sr", "Leq", "waveform", "spectrogram", "mel_spectrogram", "frequency_spectrum"]

In [5]:
for cname in category_list:
    wandb.init(
        project="Noise.Data.Analysis",
        name=datetime.datetime.now().strftime("%Y-%m-%d")+"_"+cname,
    )
    datas = []
    c = Category.from_selections(cname)
    d = MyDataset(c, base_dir)
    for i in tqdm.tqdm(range(len(c))):
        file_path = d[i * 1200][0]
        data = an.get_analysis_data(file_path)
        datas.append(data)
    
    wandb.log({cname: wandb.Table(data=datas, columns=cols)})

    wandb.finish()

100%|██████████| 20/20 [01:15<00:00,  3.79s/it]


100%|██████████| 20/20 [01:13<00:00,  3.68s/it]


100%|██████████| 20/20 [01:16<00:00,  3.82s/it]


100%|██████████| 20/20 [01:21<00:00,  4.09s/it]


100%|██████████| 20/20 [01:20<00:00,  4.01s/it]


<Figure size 640x480 with 0 Axes>